# CAMPEONATO BRASILEIRO 2020 (SÉRIE A)

In [1]:
#importando bibliotecas

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re

import dotenv
import os

## WEB SCRAPING

### Scouts defensivos

In [2]:
def defensive():
    
    """"
    Extrai os dados de scouts defensivos dos jogadores, navegando automaticamente por todas as páginas de uma url
    através do selenium. Retorna um Dataframe dos dados.
    """
    
    # iniciando a navegação da url em interesse
    
    url="https://1xbet.whoscored.com/Regions/31/Tournaments/95/Seasons/8158/Stages/18472/PlayerStatistics/Brazil-Brasileir%C3%A3o-2020"
    chrome_path="../../../chromedriver"
    driver=webdriver.Chrome(executable_path=chrome_path)
    driver.get(url)
    
    # clicando no botão com os dados de scouts defensivos
    
    botao_defensive=WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="stage-top-player-stats-options"]/li[2]/a')))
    botao_defensive.click()
    
    
    # criando um Dataframe vazio
    df=pd.DataFrame()
    
    # criando uma variável que identificará o número da página
    i=1
    
    # condição de que o número da página rode até a penúltima
    while i in range (1, 37):
        
        # esperando carregar a página atual após o clique do botão
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, f"a[data-page='{i}']")))
        
        # encontrando o elemento onde o botão que muda de página será clicado
        botao_next=WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div[5]/div[3]/div[4]/div/dl[2]/dd[3]/a')))
        
        # realizando o webscraping, adicionando ao Dataframe os dados a cada mudança de página
        
        soup=BeautifulSoup(driver.page_source, "lxml")
        tabela=soup.find_all("table")[1]
        data=[data.text.strip() for data in tabela.find_all("td")]
        colunas=[i.text for i in tabela.find_all("th")]
        numero_colunas=len(colunas)
        numero_linhas=int(len(data)/numero_colunas)
        df_temp=pd.DataFrame(np.array(data).reshape((numero_linhas, numero_colunas)), columns=colunas)
        df=df.append(df_temp)
        
        # clicando no botão que muda de página
        botao_next.click()
        
        # mudando o contador para encontrar a próxima página
        i+=1
        
        # condição para a última página
        if i==37:   
            
            # esperando carregar a página atual após o clique do botão
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, f"a[id='next'][data-page='{i}']")))
            
            # realizando o webscraping, adicionando ao Dataframe os dados a cada mudança de página

            soup=BeautifulSoup(driver.page_source, "lxml")
            tabela=soup.find_all("table")[1]
            data=[data.text.strip() for data in tabela.find_all("td")]
            colunas=[i.text for i in tabela.find_all("th")]
            numero_colunas=len(colunas)
            numero_linhas=int(len(data)/numero_colunas)
            df_temp=pd.DataFrame(np.array(data).reshape((numero_linhas, numero_colunas)), columns=colunas)
            df=df.append(df_temp)
            
            # encerrando a condição
            break
    
    # fechando o navegador
    driver.close()
    
    # retornando o Dataframe
    return df

In [3]:
# criando uma variável para o Dataframe criado
defensivo=defensive()

In [4]:
# selecionando as colunas em interesse
defensivo=defensivo.reset_index().iloc[:, np.r_[1, 3:8, 10, 13]]

# verificando o Dataframe
defensivo

,Player,Apps,Mins,Tackles,Inter,Fouls,Drb,Rating
0,"1MarinhoSantos FC, 30, AM(CLR)",20(3),1793,1.1,0.5,1.4,1,7.58
1,"2KenoAtletico MG, 31, AM(LR),FW",25(3),2211,1.2,0.6,1.2,0.9,7.49
2,"3Giorgian de ArrascaetaFlamengo, 26, AM(CLR)",17(2),1465,1.3,0.2,0.7,1,7.44
3,"4ClaudinhoRed Bull Bragantino, 23, AM(C),FW",25(2),2148,1.1,1.1,1,1.7,7.41
4,"5LucianoSao Paulo, 27, AM(CR),FW",21(2),1809,1,0.5,1.1,0.8,7.26
...,...,...,...,...,...,...,...,...
359,"360Lucho GonzálezAthletico Paranaense, 40, M(C)",4(8),342,0.5,0.2,0.8,0.5,6.13
360,"361Mariano VázquezFortaleza, 28, Midfielder",1(13),203,0.4,0.1,0.6,0.5,6.11
361,"362Jan Carlos HurtadoRed Bull Bragantino, 20, FW",2(14),286,0.1,0.1,1.2,0.6,6.07
362,"363Marcos LeonardoSantos FC, 17, Forward",3(9),297,0.2,0.1,1.2,0.3,6.04


In [5]:
# mudando o nome das colunas
defensivo.columns=["Jogador", "Jogos", "Minutos", "Roubadas_de_bola", "Desarmes", "Faltas_cometidas", "Dribles_sofridos", "Nota"]

# verificando o Dataframe
defensivo

,Jogador,Jogos,Minutos,Roubadas_de_bola,Desarmes,Faltas_cometidas,Dribles_sofridos,Nota
0,"1MarinhoSantos FC, 30, AM(CLR)",20(3),1793,1.1,0.5,1.4,1,7.58
1,"2KenoAtletico MG, 31, AM(LR),FW",25(3),2211,1.2,0.6,1.2,0.9,7.49
2,"3Giorgian de ArrascaetaFlamengo, 26, AM(CLR)",17(2),1465,1.3,0.2,0.7,1,7.44
3,"4ClaudinhoRed Bull Bragantino, 23, AM(C),FW",25(2),2148,1.1,1.1,1,1.7,7.41
4,"5LucianoSao Paulo, 27, AM(CR),FW",21(2),1809,1,0.5,1.1,0.8,7.26
...,...,...,...,...,...,...,...,...
359,"360Lucho GonzálezAthletico Paranaense, 40, M(C)",4(8),342,0.5,0.2,0.8,0.5,6.13
360,"361Mariano VázquezFortaleza, 28, Midfielder",1(13),203,0.4,0.1,0.6,0.5,6.11
361,"362Jan Carlos HurtadoRed Bull Bragantino, 20, FW",2(14),286,0.1,0.1,1.2,0.6,6.07
362,"363Marcos LeonardoSantos FC, 17, Forward",3(9),297,0.2,0.1,1.2,0.3,6.04


### Scouts ofensivos

In [10]:
def offensive():
    
    """"
    Extrai os dados de scouts ofensivos dos jogadores, navegando automaticamente por todas as páginas de uma url
    através do selenium. Retorna um Dataframe dos dados.
    """
    
    # iniciando a navegação da url em interesse
    
    url="https://1xbet.whoscored.com/Regions/31/Tournaments/95/Seasons/8158/Stages/18472/PlayerStatistics/Brazil-Brasileir%C3%A3o-2020"
    chrome_path="../../../chromedriver"
    driver=webdriver.Chrome(executable_path=chrome_path)
    driver.get(url)
    
    # clicando no botão com os dados de scouts ofensivos

    botao_offensive=WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="stage-top-player-stats-options"]/li[3]/a')))
    botao_offensive.click()
    
    # criando um Dataframe vazio
    df=pd.DataFrame()
    
    # criando uma variável que identificará o número da página
    i=1
    
    # condição de que o número da página rode até a penúltima
    while i in range (1, 37):
        
        # esperando carregar a página atual após o clique do botão
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, f"a[data-page='{i}']")))
        
        # encontrando o elemento onde o botão que muda de página será clicado
        botao_next=WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div[5]/div[4]/div[4]/div/dl[2]/dd[3]/a')))
        
        # realizando o webscraping, adicionando ao Dataframe os dados a cada mudança de página
        
        soup=BeautifulSoup(driver.page_source, "lxml")
        tabela=soup.find_all("table")[1]
        data=[data.text.strip() for data in tabela.find_all("td")]
        colunas=[i.text for i in tabela.find_all("th")]
        numero_colunas=len(colunas)
        numero_linhas=int(len(data)/numero_colunas)
        df_temp=pd.DataFrame(np.array(data).reshape((numero_linhas, numero_colunas)), columns=colunas)
        df=df.append(df_temp)
        
        # clicando no botão que muda de página
        botao_next.click()
        
        # mudando o contador para encontrar a próxima página
        i+=1
        
        # condição para a última página
        if i==37:   
            
            # esperando carregar a página atual após o clique do botão
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, f"a[id='next'][data-page='{i}']")))
               
            # realizando o webscraping, adicionando ao Dataframe os dados a cada mudança de página
            
            soup=BeautifulSoup(driver.page_source, "lxml")
            tabela=soup.find_all("table")[1]
            data=[data.text.strip() for data in tabela.find_all("td")]
            colunas=[i.text for i in tabela.find_all("th")]
            numero_colunas=len(colunas)
            numero_linhas=int(len(data)/numero_colunas)
            df_temp=pd.DataFrame(np.array(data).reshape((numero_linhas, numero_colunas)), columns=colunas)
            df=df.append(df_temp)
            
            # encerrando a condição
            break
    
    # fechando o navegador
    driver.close()
    
    # retornando o Dataframe
    return df

In [12]:
# criando uma variável para o Dataframe criado
ofensivo=offensive()

In [13]:
# selecionando as colunas em interesse
ofensivo=ofensivo.reset_index().iloc[:, np.r_[1, 5:13]]

# verificando o Dataframe
ofensivo

,Player,Goals,Assists,SpG,KeyP,Drb,Fouled,Off,Disp
0,"1MarinhoSantos FC, 30, AM(CLR)",15,6,3.3,1.1,1.6,4.7,0.8,1.7
1,"2KenoAtletico MG, 31, AM(LR),FW",10,6,3.1,1.8,3.3,2,0.9,1
2,"3Giorgian de ArrascaetaFlamengo, 26, AM(CLR)",6,5,2.4,2.8,1.5,1.5,-,1.5
3,"4ClaudinhoRed Bull Bragantino, 23, AM(C),FW",13,3,2.8,3,1.7,1.6,0.1,1.3
4,"5LucianoSao Paulo, 27, AM(CR),FW",12,3,2,1.4,1,1.8,0.3,0.9
...,...,...,...,...,...,...,...,...,...
359,"360Lucho GonzálezAthletico Paranaense, 40, M(C)",-,-,0.4,0.2,0.3,0.3,0.3,0.5
360,"361Mariano VázquezFortaleza, 28, Midfielder",-,-,0.1,0.1,0.3,0.7,0.1,0.4
361,"362Jan Carlos HurtadoRed Bull Bragantino, 20, FW",1,-,0.6,0.1,0.2,0.1,-,0.9
362,"363Marcos LeonardoSantos FC, 17, Forward",1,-,0.3,-,-,1.3,-,0.7


In [14]:
# mudando o nome das colunas
ofensivo.columns=["Jogador", "Gols", "Assistências", "Chutes", "Passes_chaves", "Dribles_realizados", "Faltas_sofridas", "Impedimentos", "Perdas_de_bola"]

# verificando o Dataframe
ofensivo

,Jogador,Gols,Assistências,Chutes,Passes_chaves,Dribles_realizados,Faltas_sofridas,Impedimentos,Perdas_de_bola
0,"1MarinhoSantos FC, 30, AM(CLR)",15,6,3.3,1.1,1.6,4.7,0.8,1.7
1,"2KenoAtletico MG, 31, AM(LR),FW",10,6,3.1,1.8,3.3,2,0.9,1
2,"3Giorgian de ArrascaetaFlamengo, 26, AM(CLR)",6,5,2.4,2.8,1.5,1.5,-,1.5
3,"4ClaudinhoRed Bull Bragantino, 23, AM(C),FW",13,3,2.8,3,1.7,1.6,0.1,1.3
4,"5LucianoSao Paulo, 27, AM(CR),FW",12,3,2,1.4,1,1.8,0.3,0.9
...,...,...,...,...,...,...,...,...,...
359,"360Lucho GonzálezAthletico Paranaense, 40, M(C)",-,-,0.4,0.2,0.3,0.3,0.3,0.5
360,"361Mariano VázquezFortaleza, 28, Midfielder",-,-,0.1,0.1,0.3,0.7,0.1,0.4
361,"362Jan Carlos HurtadoRed Bull Bragantino, 20, FW",1,-,0.6,0.1,0.2,0.1,-,0.9
362,"363Marcos LeonardoSantos FC, 17, Forward",1,-,0.3,-,-,1.3,-,0.7


### Unindo os dois Dataframes

In [15]:
# aumentando o número de colunas na visualização do Dataframe
pd.options.display.max_columns=99

In [16]:
# unindo os dois Dataframes criados com base no nome do jogador
jogadores=pd.merge(left=defensivo, right=ofensivo, on="Jogador")

# verificando o Dataframe
jogadores

,Jogador,Jogos,Minutos,Roubadas_de_bola,Desarmes,Faltas_cometidas,Dribles_sofridos,Nota,Gols,Assistências,Chutes,Passes_chaves,Dribles_realizados,Faltas_sofridas,Impedimentos,Perdas_de_bola
0,"1MarinhoSantos FC, 30, AM(CLR)",20(3),1793,1.1,0.5,1.4,1,7.58,15,6,3.3,1.1,1.6,4.7,0.8,1.7
1,"2KenoAtletico MG, 31, AM(LR),FW",25(3),2211,1.2,0.6,1.2,0.9,7.49,10,6,3.1,1.8,3.3,2,0.9,1
2,"3Giorgian de ArrascaetaFlamengo, 26, AM(CLR)",17(2),1465,1.3,0.2,0.7,1,7.44,6,5,2.4,2.8,1.5,1.5,-,1.5
3,"4ClaudinhoRed Bull Bragantino, 23, AM(C),FW",25(2),2148,1.1,1.1,1,1.7,7.41,13,3,2.8,3,1.7,1.6,0.1,1.3
4,"5LucianoSao Paulo, 27, AM(CR),FW",21(2),1809,1,0.5,1.1,0.8,7.26,12,3,2,1.4,1,1.8,0.3,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,"360Lucho GonzálezAthletico Paranaense, 40, M(C)",4(8),342,0.5,0.2,0.8,0.5,6.13,-,-,0.4,0.2,0.3,0.3,0.3,0.5
360,"361Mariano VázquezFortaleza, 28, Midfielder",1(13),203,0.4,0.1,0.6,0.5,6.11,-,-,0.1,0.1,0.3,0.7,0.1,0.4
361,"362Jan Carlos HurtadoRed Bull Bragantino, 20, FW",2(14),286,0.1,0.1,1.2,0.6,6.07,1,-,0.6,0.1,0.2,0.1,-,0.9
362,"363Marcos LeonardoSantos FC, 17, Forward",3(9),297,0.2,0.1,1.2,0.3,6.04,1,-,0.3,-,-,1.3,-,0.7


## LIMPEZA E MANIPULAÇÃO DOS DADOS

In [17]:
# criando funções para separar o nome, time e idade do jogador, através de expressões regulare

def jogador(x):
    """
    Retorna o nome do jogador
    """
    pattern="\d*(.*[a-záéêíóôú])[A-Z].*, \d*, .*"
    captura_nome=re.findall(pattern, x)[0]
    return captura_nome

def time(x):
    """
    Retorna o time do jogador
    """
    pattern="\d*.*[a-záéêíóôú]([A-Z].*), \d*, .*"
    captura_time=re.findall(pattern, x)[0]
    return captura_time

def idade(x):
    """
    Retorna a idade do jogador
    """
    pattern="\d*.*, (\d*), .*"
    captura_idade=re.findall(pattern, x)[0]
    return captura_idade

In [18]:
# aplicando as funções criadas no Dataframe, criando uma coluna para cada informação

jogadores["Jogadores"]=jogadores["Jogador"].apply(jogador)
jogadores["Time"]=jogadores["Jogador"].apply(time)
jogadores["Idade"]=jogadores["Jogador"].apply(idade)

# verificando o Dataframe
jogadores

,Jogador,Jogos,Minutos,Roubadas_de_bola,Desarmes,Faltas_cometidas,Dribles_sofridos,Nota,Gols,Assistências,Chutes,Passes_chaves,Dribles_realizados,Faltas_sofridas,Impedimentos,Perdas_de_bola,Jogadores,Time,Idade
0,"1MarinhoSantos FC, 30, AM(CLR)",20(3),1793,1.1,0.5,1.4,1,7.58,15,6,3.3,1.1,1.6,4.7,0.8,1.7,Marinho,Santos FC,30
1,"2KenoAtletico MG, 31, AM(LR),FW",25(3),2211,1.2,0.6,1.2,0.9,7.49,10,6,3.1,1.8,3.3,2,0.9,1,Keno,Atletico MG,31
2,"3Giorgian de ArrascaetaFlamengo, 26, AM(CLR)",17(2),1465,1.3,0.2,0.7,1,7.44,6,5,2.4,2.8,1.5,1.5,-,1.5,Giorgian de Arrascaeta,Flamengo,26
3,"4ClaudinhoRed Bull Bragantino, 23, AM(C),FW",25(2),2148,1.1,1.1,1,1.7,7.41,13,3,2.8,3,1.7,1.6,0.1,1.3,Claudinho,Red Bull Bragantino,23
4,"5LucianoSao Paulo, 27, AM(CR),FW",21(2),1809,1,0.5,1.1,0.8,7.26,12,3,2,1.4,1,1.8,0.3,0.9,Luciano,Sao Paulo,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,"360Lucho GonzálezAthletico Paranaense, 40, M(C)",4(8),342,0.5,0.2,0.8,0.5,6.13,-,-,0.4,0.2,0.3,0.3,0.3,0.5,Lucho González,Athletico Paranaense,40
360,"361Mariano VázquezFortaleza, 28, Midfielder",1(13),203,0.4,0.1,0.6,0.5,6.11,-,-,0.1,0.1,0.3,0.7,0.1,0.4,Mariano Vázquez,Fortaleza,28
361,"362Jan Carlos HurtadoRed Bull Bragantino, 20, FW",2(14),286,0.1,0.1,1.2,0.6,6.07,1,-,0.6,0.1,0.2,0.1,-,0.9,Jan Carlos Hurtado,Red Bull Bragantino,20
362,"363Marcos LeonardoSantos FC, 17, Forward",3(9),297,0.2,0.1,1.2,0.3,6.04,1,-,0.3,-,-,1.3,-,0.7,Marcos Leonardo,Santos FC,17


In [19]:
# dropando a coluna com as informações juntas
jogadores=jogadores.drop(columns="Jogador")

#verificando o Dataframe
jogadores

,Jogos,Minutos,Roubadas_de_bola,Desarmes,Faltas_cometidas,Dribles_sofridos,Nota,Gols,Assistências,Chutes,Passes_chaves,Dribles_realizados,Faltas_sofridas,Impedimentos,Perdas_de_bola,Jogadores,Time,Idade
0,20(3),1793,1.1,0.5,1.4,1,7.58,15,6,3.3,1.1,1.6,4.7,0.8,1.7,Marinho,Santos FC,30
1,25(3),2211,1.2,0.6,1.2,0.9,7.49,10,6,3.1,1.8,3.3,2,0.9,1,Keno,Atletico MG,31
2,17(2),1465,1.3,0.2,0.7,1,7.44,6,5,2.4,2.8,1.5,1.5,-,1.5,Giorgian de Arrascaeta,Flamengo,26
3,25(2),2148,1.1,1.1,1,1.7,7.41,13,3,2.8,3,1.7,1.6,0.1,1.3,Claudinho,Red Bull Bragantino,23
4,21(2),1809,1,0.5,1.1,0.8,7.26,12,3,2,1.4,1,1.8,0.3,0.9,Luciano,Sao Paulo,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,4(8),342,0.5,0.2,0.8,0.5,6.13,-,-,0.4,0.2,0.3,0.3,0.3,0.5,Lucho González,Athletico Paranaense,40
360,1(13),203,0.4,0.1,0.6,0.5,6.11,-,-,0.1,0.1,0.3,0.7,0.1,0.4,Mariano Vázquez,Fortaleza,28
361,2(14),286,0.1,0.1,1.2,0.6,6.07,1,-,0.6,0.1,0.2,0.1,-,0.9,Jan Carlos Hurtado,Red Bull Bragantino,20
362,3(9),297,0.2,0.1,1.2,0.3,6.04,1,-,0.3,-,-,1.3,-,0.7,Marcos Leonardo,Santos FC,17


In [20]:
# verificando as colunas
jogadores.columns

Index(['Jogos', 'Minutos', 'Roubadas_de_bola', 'Desarmes', 'Faltas_cometidas',
       'Dribles_sofridos', 'Nota', 'Gols', 'Assistências', 'Chutes',
       'Passes_chaves', 'Dribles_realizados', 'Faltas_sofridas',
       'Impedimentos', 'Perdas_de_bola', 'Jogadores', 'Time', 'Idade'],
      dtype='object')

In [21]:
# ordenando as colunas
jogadores=jogadores[['Jogadores', 'Time', 'Idade', 'Jogos', 'Minutos', 'Roubadas_de_bola', 'Desarmes', 'Faltas_cometidas',
       'Dribles_sofridos', 'Gols', 'Assistências', 'Chutes',
       'Passes_chaves', 'Dribles_realizados', 'Faltas_sofridas',
       'Impedimentos', 'Perdas_de_bola', 'Nota']]

# verificando o Dataframe
jogadores

,Jogadores,Time,Idade,Jogos,Minutos,Roubadas_de_bola,Desarmes,Faltas_cometidas,Dribles_sofridos,Gols,Assistências,Chutes,Passes_chaves,Dribles_realizados,Faltas_sofridas,Impedimentos,Perdas_de_bola,Nota
0,Marinho,Santos FC,30,20(3),1793,1.1,0.5,1.4,1,15,6,3.3,1.1,1.6,4.7,0.8,1.7,7.58
1,Keno,Atletico MG,31,25(3),2211,1.2,0.6,1.2,0.9,10,6,3.1,1.8,3.3,2,0.9,1,7.49
2,Giorgian de Arrascaeta,Flamengo,26,17(2),1465,1.3,0.2,0.7,1,6,5,2.4,2.8,1.5,1.5,-,1.5,7.44
3,Claudinho,Red Bull Bragantino,23,25(2),2148,1.1,1.1,1,1.7,13,3,2.8,3,1.7,1.6,0.1,1.3,7.41
4,Luciano,Sao Paulo,27,21(2),1809,1,0.5,1.1,0.8,12,3,2,1.4,1,1.8,0.3,0.9,7.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,Lucho González,Athletico Paranaense,40,4(8),342,0.5,0.2,0.8,0.5,-,-,0.4,0.2,0.3,0.3,0.3,0.5,6.13
360,Mariano Vázquez,Fortaleza,28,1(13),203,0.4,0.1,0.6,0.5,-,-,0.1,0.1,0.3,0.7,0.1,0.4,6.11
361,Jan Carlos Hurtado,Red Bull Bragantino,20,2(14),286,0.1,0.1,1.2,0.6,1,-,0.6,0.1,0.2,0.1,-,0.9,6.07
362,Marcos Leonardo,Santos FC,17,3(9),297,0.2,0.1,1.2,0.3,1,-,0.3,-,-,1.3,-,0.7,6.04


In [22]:
# substituindo os valores "-" por 0 e renomeando a coluna dos jogadores
jogadores=jogadores.replace("-", 0).rename({"Jogadores": "Jogador"}, axis=1)

# verificando o Dataframe
jogadores

,Jogador,Time,Idade,Jogos,Minutos,Roubadas_de_bola,Desarmes,Faltas_cometidas,Dribles_sofridos,Gols,Assistências,Chutes,Passes_chaves,Dribles_realizados,Faltas_sofridas,Impedimentos,Perdas_de_bola,Nota
0,Marinho,Santos FC,30,20(3),1793,1.1,0.5,1.4,1,15,6,3.3,1.1,1.6,4.7,0.8,1.7,7.58
1,Keno,Atletico MG,31,25(3),2211,1.2,0.6,1.2,0.9,10,6,3.1,1.8,3.3,2,0.9,1,7.49
2,Giorgian de Arrascaeta,Flamengo,26,17(2),1465,1.3,0.2,0.7,1,6,5,2.4,2.8,1.5,1.5,0,1.5,7.44
3,Claudinho,Red Bull Bragantino,23,25(2),2148,1.1,1.1,1,1.7,13,3,2.8,3,1.7,1.6,0.1,1.3,7.41
4,Luciano,Sao Paulo,27,21(2),1809,1,0.5,1.1,0.8,12,3,2,1.4,1,1.8,0.3,0.9,7.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,Lucho González,Athletico Paranaense,40,4(8),342,0.5,0.2,0.8,0.5,0,0,0.4,0.2,0.3,0.3,0.3,0.5,6.13
360,Mariano Vázquez,Fortaleza,28,1(13),203,0.4,0.1,0.6,0.5,0,0,0.1,0.1,0.3,0.7,0.1,0.4,6.11
361,Jan Carlos Hurtado,Red Bull Bragantino,20,2(14),286,0.1,0.1,1.2,0.6,1,0,0.6,0.1,0.2,0.1,0,0.9,6.07
362,Marcos Leonardo,Santos FC,17,3(9),297,0.2,0.1,1.2,0.3,1,0,0.3,0,0,1.3,0,0.7,6.04


In [23]:
# verificando jogadores com o mesmo nome
jogadores.loc[jogadores["Jogador"].duplicated(keep=False), :].sort_values(by="Jogador")

,Jogador,Time,Idade,Jogos,Minutos,Roubadas_de_bola,Desarmes,Faltas_cometidas,Dribles_sofridos,Gols,Assistências,Chutes,Passes_chaves,Dribles_realizados,Faltas_sofridas,Impedimentos,Perdas_de_bola,Nota
7,Bruno Henrique,Flamengo,30,22(1),1888,0.9,0.5,1.5,0.3,6,6,2.7,1.3,1.3,2.5,0.3,1.4,7.22
293,Bruno Henrique,Palmeiras,31,7(6),630,0.8,0.5,1.3,0.4,0,0,0.8,0.8,0.5,0.5,0.1,0.2,6.44
192,Carlinhos,Fortaleza,34,16(6),1547,1,1,1.4,0.2,0,0,0.2,0.8,0.1,1,0.1,0.2,6.63
214,Carlinhos,Vasco da Gama,26,10(9),891,1.3,0.3,1,0.7,1,1,1.1,0.9,1.1,1.1,0,0.5,6.60
24,Diego,Sao Paulo,21,17,1417,1.7,1.6,1.9,0.5,1,0,0.5,0.2,0.6,0.8,0,0.1,7.07
134,Diego,Flamengo,35,6(12),671,0.8,0.3,1,1.1,1,3,0.7,0.9,1.3,2.2,0,0.4,6.71
257,Fabinho,Athletico Paranaense,21,6(7),700,0.8,0.2,0.3,0.5,1,0,1.2,0.9,1.4,0.3,0.2,0.8,6.54
263,Fabinho,Ceara,34,20(5),1775,1.1,1.7,0.4,0.9,0,0,0.4,0.2,0.5,0.6,0,0.4,6.52
39,Juninho,Fortaleza,34,25(1),2235,1.7,1.3,1.5,1,2,3,1.4,1.7,1.1,1.5,0,0.6,6.97
272,Juninho,Bahia,25,23,2100,1,1.3,0.2,0.5,2,0,0.6,0.2,0.2,0.4,0,0.1,6.49


In [24]:
# criando uma coluna com o index dos jogadores, chamando-a de ID
jogadores=jogadores.reset_index().rename({"index": "ID"}, axis=1)

# verificando o Datafram
jogadores

,ID,Jogador,Time,Idade,Jogos,Minutos,Roubadas_de_bola,Desarmes,Faltas_cometidas,Dribles_sofridos,Gols,Assistências,Chutes,Passes_chaves,Dribles_realizados,Faltas_sofridas,Impedimentos,Perdas_de_bola,Nota
0,0,Marinho,Santos FC,30,20(3),1793,1.1,0.5,1.4,1,15,6,3.3,1.1,1.6,4.7,0.8,1.7,7.58
1,1,Keno,Atletico MG,31,25(3),2211,1.2,0.6,1.2,0.9,10,6,3.1,1.8,3.3,2,0.9,1,7.49
2,2,Giorgian de Arrascaeta,Flamengo,26,17(2),1465,1.3,0.2,0.7,1,6,5,2.4,2.8,1.5,1.5,0,1.5,7.44
3,3,Claudinho,Red Bull Bragantino,23,25(2),2148,1.1,1.1,1,1.7,13,3,2.8,3,1.7,1.6,0.1,1.3,7.41
4,4,Luciano,Sao Paulo,27,21(2),1809,1,0.5,1.1,0.8,12,3,2,1.4,1,1.8,0.3,0.9,7.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,359,Lucho González,Athletico Paranaense,40,4(8),342,0.5,0.2,0.8,0.5,0,0,0.4,0.2,0.3,0.3,0.3,0.5,6.13
360,360,Mariano Vázquez,Fortaleza,28,1(13),203,0.4,0.1,0.6,0.5,0,0,0.1,0.1,0.3,0.7,0.1,0.4,6.11
361,361,Jan Carlos Hurtado,Red Bull Bragantino,20,2(14),286,0.1,0.1,1.2,0.6,1,0,0.6,0.1,0.2,0.1,0,0.9,6.07
362,362,Marcos Leonardo,Santos FC,17,3(9),297,0.2,0.1,1.2,0.3,1,0,0.3,0,0,1.3,0,0.7,6.04


## PostgreSQL

### Ajustes

In [25]:
# carregando o arquivo .env que contém a senha do SQL
dotenv.load_dotenv("../headers.env")

True

In [26]:
# importando biblioteca
import sqlalchemy as db

# variáveis necessárias
servidor="postgresql"
usuário="postgres"
senha=os.getenv("pg_admin")
ip="localhost"
nome="Jogadores - Série A"

# criando diretório
diretório=db.create_engine(f"{servidor}://{usuário}:{senha}@{ip}/{nome}")

# abrindo a conexão
conexão=diretório.connect()

### Expotando o Dataframe

In [27]:
# exportando o Dataframe para o SQL
jogadores.to_sql("jogadores", conexão, if_exists="replace", index=False)

# exportando o Dataframe para um arquivo csv
jogadores.to_csv("./data/jogadores.csv", index=False)